<a href="https://colab.research.google.com/github/rojinadeuja/NLP-Model-Implementations/blob/master/GloVe_using_Keras_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Modules

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

## Load Dataset

In [3]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/IMDB.csv')
df.replace(['positive', 'negative'], [1, 0], inplace=True)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


## Data Pre-processing

In [4]:
def preprocess(s):
    '''Function for data pre-processing'''
    # Removing html tags
    TAG_RE = re.compile(r'<[^>]+>')
    sentence = TAG_RE.sub('', s)
    
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

## Create X and y matrices

In [5]:
X = []
sentences = list(df['review'])
for sentence in sentences:
    X.append(preprocess(sentence))

y = df['sentiment']

## Split dataset into train and test

In [7]:
# Create train-test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(len(X_train), len(X_test), len(y_train), len(y_test))

40000 10000 40000 10000


In [8]:
# Convert into numpy arrays for processing with tensorflo
y_train = np.array(y_train)
y_test = np.array(y_test)
print(type(X_train), type(X_test), type(y_train), type(y_test))

<class 'list'> <class 'list'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


## Tokenize the data

In [9]:
# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [12]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

## Create Embeddings

In [13]:
# Create embeddings
embeddings_dictionary = dict()
glove_file = open('/content/drive/My Drive/Colab Notebooks/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [14]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

## Classification using a simple Neural Network

In [15]:
# Text Classification with a Simple Neural Network
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          9254700   
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 10001     
Total params: 9,264,701
Trainable params: 10,001
Non-trainable params: 9,254,700
_________________________________________________________________
None


## Train the model

In [17]:
# Train the model
history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

Epoch 1/6
250/250 [==============================] - 1s 4ms/step - loss: 0.6035 - acc: 0.6714 - val_loss: 0.5320 - val_acc: 0.7360
Epoch 2/6
250/250 [==============================] - 1s 4ms/step - loss: 0.4976 - acc: 0.7602 - val_loss: 0.5187 - val_acc: 0.7458
Epoch 3/6
250/250 [==============================] - 1s 4ms/step - loss: 0.4615 - acc: 0.7814 - val_loss: 0.5161 - val_acc: 0.7500
Epoch 4/6
250/250 [==============================] - 1s 4ms/step - loss: 0.4393 - acc: 0.7947 - val_loss: 0.5385 - val_acc: 0.7370
Epoch 5/6
250/250 [==============================] - 1s 4ms/step - loss: 0.4221 - acc: 0.8065 - val_loss: 0.5296 - val_acc: 0.7464
Epoch 6/6
250/250 [==============================] - 1s 4ms/step - loss: 0.4075 - acc: 0.8163 - val_loss: 0.5346 - val_acc: 0.7418


## Evaluate model on test data

In [18]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=1)
print("\nTest Accuracy:", score[1])

313/313 [==============================] - 0s 1ms/step - loss: 0.5357 - acc: 0.7450

Test Accuracy: 0.7450000047683716
